In [271]:
import random
import networkx as nx
import math

In [272]:
try:
    gph = nx.read_gexf("./microGraph.gexf")
    print("graph file found at " + str())
    print(gph)

except Exception as e: # if any error
    print(e)
    gph = nx.Graph()

graph file found at 
Graph with 500 nodes and 2502 edges


In [273]:
values = {}
contributions = set() # all nodes with type contribution
starred = set() # all nodes with type starred repo


In [274]:
for n in gph.nodes(data = True):
    if n[1]['bipartite'] == 0:
        values[n[0]] = 1
    else:
        values[n[0]] = 2

for n in gph.edges(data = True):
    if 'contributions' in n[2]:
        contributions.add(n[0])
        contributions.add(n[1])
    else:
        starred.add(n[0])
        starred.add(n[1])

In [275]:
threshold = 3

def algo(gph):
    predicted = []

    for node in gph.nodes():

        if values[node] == 1: # if it is user
            current_user = node
            weight = {} # to store weights of issues
            prediction = []
            for nodes in gph.neighbors(current_user):
                if values[nodes] == 2: # issues connected to users
                    issues = nodes
                    for issue_neigh in gph.neighbors(issues): # neighbours of issues connected to users
                        if values[issue_neigh] == 2:
                            if issue_neigh in weight:
                                weight[issue_neigh] += 1
                            else:
                                weight[issue_neigh] = 1
                else: # users connected to users
                    user = nodes
                    if user in contributions: # if user is connected via contribution
                        for issue_neigh in gph.neighbors(user): # issues that are neighbours of users connected to cuurent user 
                            if values[issue_neigh] == 2:
                                if issue_neigh in weight:
                                    weight[issue_neigh] += 2
                                else:
                                    weight[issue_neigh] = 2
                    else: # if user is connected via starred repo
                        for issue_neigh in gph.neighbors(user): # issues that are neighbours of users connected to cuurent user
                            if values[issue_neigh] == 2:
                                if issue_neigh in weight:
                                    weight[issue_neigh] += 1
                                else:
                                    weight[issue_neigh] = 1
            
            Issues = list(weight.keys())
            Weights = list(weight.values())

            # print(Issues)
            # print(Weights)

            # print(current_user, end = " --> ")
            # Probability = Element Weight / Sum Of All Weights
            
            # for i in Issues:
            #     for j in Weights:
            #         if (i != None and j!= None):

            sorted_prediction = sorted(weight.items(), key = lambda x:x[1], reverse=True)
            predicted_issue = sorted_prediction[0][0]
            # predicted_issue = random.choices(Issues, Weights, k = 1)[0]
            # print(predicted_issue) # k is the number of outputs
            
            predicted.append([current_user, predicted_issue])
    
    return predicted


In [276]:
B= gph


users = [n for n, d in B.nodes(data=True) if d["bipartite"] == 0]
issues = [n for n, d in B.nodes(data=True) if d["bipartite"] == 1]

n = len(issues)

In [277]:
def removeEdges(graph):
    g = nx.Graph()
    initial = graph.number_of_edges()
    issuesList = random.sample(issues, 10)
    for i in issuesList:
        for j in list(graph.neighbors(i)):
            if graph.nodes[j]["bipartite"] == 0:
                g.add_node(i, bipartite=1)
                g.add_node(j, bipartite=0)
                g.add_edge(i,j)
                graph.remove_edge(i,j)
    final = graph.number_of_edges()
    print(initial - final)
    return g,graph

train, test = removeEdges(B)

10


In [278]:
predicted = algo(test)

   

IndexError: list index out of range

In [ ]:
og_list = train.edges()
og_list = list(og_list)

print(og_list)
common_edges = []

for edge1 in predicted:
    for edge2 in og_list:
        sorted_edge1 = tuple(sorted(edge1))
        sorted_edge2 = tuple(sorted(edge2))
        if sorted_edge1 == sorted_edge2:
            common_edges.append(edge1)
            break

print(len(common_edges))
for i in common_edges:
    print(i)


[('PR_kwDOCmABf85CSdka', '0xZOne'), ('MDExOlB1bGxSZXF1ZXN0MzI1ODI1MjU2', 'apomalyn'), ('PR_kwDOBy2NM85I5ljj', 'hbina'), ('hbina', 'PR_kwDOBy2NM85I342w'), ('MDExOlB1bGxSZXF1ZXN0MzYwNjEwMzIw', 'brianegan'), ('PR_kwDOAiEj5s5IyevW', 'dart-lang'), ('MDExOlB1bGxSZXF1ZXN0MjY2OTgzMjY3', 'hutcwp'), ('PR_kwDODBDVB85KB3_p', 'firebase'), ('PR_kwDOCmABf85FQ_fS', 'JunhuaLin'), ('MDExOlB1bGxSZXF1ZXN0MzYyNTE3MzQ4', 'htynkn')]
0
